# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("analysis_data.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Avarua,93,CK,1600152016,68,-21.21,-159.78,71.60,6.93
1,Tilichiki,96,RU,1600152016,73,60.47,166.10,44.76,9.80
2,Hay River,75,CA,1600151799,87,60.82,-115.80,42.80,6.93
3,Arman',100,RU,1600152017,77,59.70,150.17,53.55,5.44
4,Provideniya,75,RU,1600152017,71,64.38,-173.30,50.00,11.18
...,...,...,...,...,...,...,...,...,...
570,Evensk,99,RU,1600152246,69,61.95,159.23,47.75,6.42
571,Inta,9,RU,1600152246,70,66.03,60.17,53.44,8.48
572,Tiznit Province,100,MA,1600152246,45,29.58,-9.50,73.90,1.57
573,Muros,71,ES,1600152247,84,42.78,-9.06,66.00,7.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
123,Guerrero Negro,0,MX,1600152052,52,27.98,-114.06,77.85,8.59
160,Kropotkin,0,RU,1600152062,38,45.44,40.58,74.62,6.35
184,Kabinda,0,CD,1600152073,64,-6.13,24.48,78.84,5.77
247,Pahrump,0,US,1600152088,24,36.21,-115.98,78.01,5.01
267,Bījār,0,IR,1600152094,21,35.87,47.61,73.81,5.59
304,Knin,0,HR,1600152103,48,44.04,16.20,75.99,3.00
362,Nortelândia,0,BR,1600152117,33,-14.45,-56.80,71.28,3.27
367,Lazurne,0,UA,1600152118,49,46.08,32.53,73.00,6.71
429,Kardamena,0,GR,1600152209,74,36.78,27.14,78.80,5.82
461,Óbidos,0,BR,1600152217,82,-1.92,-55.52,75.40,3.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
123,Guerrero Negro,MX,27.98,-114.06,
160,Kropotkin,RU,45.44,40.58,
184,Kabinda,CD,-6.13,24.48,
247,Pahrump,US,36.21,-115.98,
267,Bījār,IR,35.87,47.61,
304,Knin,HR,44.04,16.20,
362,Nortelândia,BR,-14.45,-56.80,
367,Lazurne,UA,46.08,32.53,
429,Kardamena,GR,36.78,27.14,
461,Óbidos,BR,-1.92,-55.52,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 123: Guerrero Negro.
Closest hotel in Guerrero Negro is Hotel TerraSal.
------------
Retrieving Results for Index 160: Kropotkin.
Closest hotel in Kropotkin is Luxury House ECO-hotel & Spa.
------------
Retrieving Results for Index 184: Kabinda.
Missing field/result... skipping.
------------
Retrieving Results for Index 247: Pahrump.
Closest hotel in Pahrump is Holiday Inn Express & Suites Pahrump.
------------
Retrieving Results for Index 267: Bījār.
Closest hotel in Bījār is Hotel Bame Iran.
------------
Retrieving Results for Index 304: Knin.
Closest hotel in Knin is Hotel Mihovil.
------------
Retrieving Results for Index 362: Nortelândia.
Closest hotel in Nortelândia is Hotel Karysmã.
------------
Retrieving Results for Index 367: Lazurne.
Closest hotel in Lazurne is Granat.
------------
Retrieving Results for Index 429: Kardamena.
Closest hotel in Kardamena is Mitsis Blue Domes Resort & Spa.
------------
Retrieving Results for Index 461: Óbidos.
Close

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
123,Guerrero Negro,MX,27.98,-114.06,Hotel TerraSal
160,Kropotkin,RU,45.44,40.58,Luxury House ECO-hotel & Spa
184,Kabinda,CD,-6.13,24.48,
247,Pahrump,US,36.21,-115.98,Holiday Inn Express & Suites Pahrump
267,Bījār,IR,35.87,47.61,Hotel Bame Iran
304,Knin,HR,44.04,16.20,Hotel Mihovil
362,Nortelândia,BR,-14.45,-56.80,Hotel Karysmã
367,Lazurne,UA,46.08,32.53,Granat
429,Kardamena,GR,36.78,27.14,Mitsis Blue Domes Resort & Spa
461,Óbidos,BR,-1.92,-55.52,Hotel Amazônia Center


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))